In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
file_dir = "/ifs/projects/proj052/pipeline_proj052/flow_tables.dir/"
tables = [x for x in os.listdir(file_dir) if re.search("fano", x)]

In [3]:
tab1 = tables[0]
df = pd.read_table(file_dir + tab1, header=0, index_col=None, sep="\t")
tables.remove(tab1)
for tab in tables:
    _df = pd.read_table(file_dir + tab, header=0, index_col=None, sep="\t")
    df = df.append(_df)

In [4]:
df["twin_num"] = [int(xi.split("_")[-1]) for xi in df["twin.id"].values]

In [5]:
demo_file = "/ifs/projects/proj052/backup/Twin_pairs.csv"
demo_df = pd.read_table(demo_file, sep="\t", header=0, index_col=None)

In [6]:
demo_df.head()

,FlowJo Sample ID,Unique Family ID,Zygosity,Age,Replicate#,Visit#
0,1,23,DZ,65.45,1,1
1,2,23,DZ,65.45,1,1
2,3,15,DZ,66.63,1,1
3,4,15,DZ,66.63,1,1
4,5,65,DZ,61.67,1,1


In [7]:
merge_df = pd.merge(left=df, right=demo_df, left_on="twin_num", right_on="FlowJo Sample ID", how='inner')
#merge_df.index = merge_df["twin.id"]
#merge_df.drop(["twin.id"], axis=1, inplace=True)

In [8]:
merge_df.head()
cols = ["NA1", "NA2", "NA3", "CD161", "NA4", "PD-1", "Aq_blue", "NA5", "CD8", "CCR6", "CD45RA", "NA6", "CD4", 
        "CXCR5", "CCR7", "NA7", "CCR10", "CD3", "CXCR3", "NA8", "CCR4", "NA9", "twin.id", "subset", "twin_num", 
        "flowjo_id", "family_id", "zygosity", "age", "replicate", "visit"]
merge_df.columns = cols

In [9]:
#merge_df.to_csv("/ifs/projects/proj052/flow_processing_tables/Twins_merged_data.tsv", sep="\t", index_label="indx")

In [10]:
# split into MZ and DZ - check each individual matches up with another
MZ_df = merge_df[merge_df["zygosity"] == "MZ"]
DZ_df = merge_df[merge_df["zygosity"] == "DZ"]

In [11]:
MZ_1 = MZ_df[MZ_df.duplicated(subset="family_id")]
MZ_2 = MZ_df[[not i for i in MZ_df.duplicated(subset="family_id")]]

In [12]:
MZ1_melt = pd.melt(MZ_1, id_vars=["twin.id", "twin_num", "family_id", "flowjo_id", "zygosity", "age", "subset",
                                  "replicate", "visit"], var_name="marker", value_name="twin1")
MZ2_melt = pd.melt(MZ_2, id_vars=["twin.id", "twin_num", "family_id", "flowjo_id", "zygosity", "age", "subset",
                                  "replicate", "visit"], var_name="marker", value_name="twin2")

In [13]:
matched_MZ = pd.merge(left=MZ1_melt, right=MZ2_melt, on=["marker", "zygosity", "age", "family_id"], how='inner')

In [14]:
#matched_MZ.to_csv("/ifs/projects/proj052/flow_processing_tables/MZ_twins_data.tsv", sep="\t")

In [15]:
# and now for DZ twins
DZ_1 = DZ_df[DZ_df.duplicated(subset="family_id")]
DZ_2 = DZ_df[[not i for i in DZ_df.duplicated(subset="family_id")]]

In [24]:
DZ1_melt = pd.melt(DZ_1, id_vars=["twin.id", "twin_num", "family_id", "flowjo_id", "zygosity", "age", "subset",
                                  "replicate", "visit"], var_name="marker", value_name="twin1")
DZ2_melt = pd.melt(DZ_2, id_vars=["twin.id", "twin_num", "family_id", "flowjo_id", "zygosity", "age", "subset",
                                  "replicate", "visit"], var_name="marker", value_name="twin2")

In [25]:
matched_DZ = pd.merge(left=DZ1_melt, right=DZ2_melt, on=["marker", "zygosity", "age", "family_id"], how='inner')

In [26]:
#matched_DZ.to_csv("/ifs/projects/proj052/flow_processing_tables/DZ_twins_data.tsv", sep="\t")


In [27]:
# subset into multiple dfs, one for each marker
DZ_groups = matched_DZ.groupby(['marker'])
MZ_groups = matched_MZ.groupby(['marker'])

In [28]:
# iterate through markers, calculating twin (intra-class) correlations
DZ_cor_dict = {}
for dname, dgroup in DZ_groups:
    dztwin_cor = np.corrcoef(dgroup['twin1'], dgroup['twin2'])
    DZ_cor_dict[dname] = dztwin_cor[0, 1]
    
MZ_cor_dict = {}
for mname, mgroup in MZ_groups:
    mztwin_cor = np.corrcoef(mgroup['twin1'], mgroup['twin2'])  
    MZ_cor_dict[mname] = mztwin_cor[0, 1]

In [29]:
mztwin_cor[0, 1], dztwin_cor[0, 1]

(0.47491308662266413, 0.41865395806363487)

In [34]:
# calculate heritability as 2x the twin-class correlation difference
broad_h = {}
for key in MZ_cor_dict.keys():
    if not re.search("NA", key):
        mz_cor = MZ_cor_dict[key]
        dz_cor = DZ_cor_dict[key]
        H2 = 2 * (mz_cor - dz_cor)
        broad_h[key] = H2
    else:
        pass

In [35]:
pd.Series(broad_h)

Aq_blue   -0.105993
CCR10     -0.146619
CCR4       0.483622
CCR6       0.428939
CCR7       0.111381
CD161      0.250414
CD3        0.271304
CD4       -0.147852
CD45RA     0.357381
CD8       -0.123898
CXCR3     -0.900234
CXCR5     -0.094029
PD-1       0.112518
dtype: float64